In [3]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Set plot style
sns.set(style='whitegrid')
%matplotlib inline


In [4]:
data_path=os.path.join('../data/pilot_data/','sub-01_ses-001_raw_preprocessed_GonoGo_noEpoched.fif')
raw=mne.io.read_raw_fif(data_path,preload=True)

Opening raw data file ../data/pilot_data/sub-01_ses-001_raw_preprocessed_GonoGo_noEpoched.fif...
    Range : 81028 ... 385484 =    270.093 ...  1284.947 secs
Ready.


C:\Users\s\AppData\Local\Temp\ipykernel_5344\1495992042.py:2: RuntimeWarning: This filename (../data/pilot_data/sub-01_ses-001_raw_preprocessed_GonoGo_noEpoched.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw=mne.io.read_raw_fif(data_path,preload=True)


Reading 0 ... 304456  =      0.000 ...  1014.853 secs...


In [6]:
# Define your event IDs based on the trigger list
event_id = {
    'Go': 1,
    'NoGo': 2,
    'Correct': 3,
    'Incorrect': 4
}

# Replace 'STI 014' with your actual stimulus channel name if different
events = mne.find_events(raw, stim_channel='Trigger')

# Display the first few events
print(events[:10])
# Initialize lists to store new events and their IDs
new_events = []
new_event_id = {
    'Go_Correct': 101,
    'Go_Incorrect': 102,
    'NoGo_Correct': 201,
    'NoGo_Incorrect': 202
}

# Iterate through events to combine onset and response triggers
i = 0
while i < len(events) - 1:
    onset_event = events[i]
    response_event = events[i + 1]
    
    onset = onset_event[2]
    response = response_event[2]
    
    # Check if the current event is an onset trigger and the next is a response trigger
    if onset in [1, 2] and response in [3, 4]:
        if onset == 1 and response == 3:
            new_events.append([onset_event[0], 0, new_event_id['Go_Correct']])
        elif onset == 1 and response == 4:
            new_events.append([onset_event[0], 0, new_event_id['Go_Incorrect']])
        elif onset == 2 and response == 3:
            new_events.append([onset_event[0], 0, new_event_id['NoGo_Correct']])
        elif onset == 2 and response == 4:
            new_events.append([onset_event[0], 0, new_event_id['NoGo_Incorrect']])
        # Skip the response event in the next iteration
        i += 2
    else:
        # If not a valid onset-response pair, skip to the next event
        i += 1

# Convert to numpy array
new_events = np.array(new_events)

# Display the first few new events
print(new_events[:10])


Trigger channel Trigger has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
1198 events found on stim channel Trigger
Event IDs: [1 2 3 4 8 9]
[[81043     7     8]
 [81636     0     2]
 [82826     0     3]
 [82982     0     1]
 [83194     0     1]
 [83490     0     3]
 [83648     0     1]
 [83821     0     1]
 [84117     0     3]
 [84276     0     1]]
[[81636     0   201]
 [83194     0   101]
 [83821     0   101]
 [84409     0   101]
 [84865     0   201]
 [86346     0   101]
 [86927     0   101]
 [87498     0   101]
 [88848     0   201]
 [89438     0   101]]


In [ ]:
# ----------------------------
# Assing Events to the tasks
#-----------------------------

# Function to determine which task a sample belongs to
def get_task_for_sample(sample, task_periods):
    for task_name, period in task_periods.items():
        if period['start'] <= sample <= period['end']:
            return task_name
    return None  # Not within any task period

# Initialize a list to store task-specific events
task_specific_events = []

# Iterate through all events and assign to tasks
for event in events:
    sample, _, trigger = event
    task = get_task_for_sample(sample, task_periods)
    
    # Skip events outside defined tasks
    if task is None:
        continue
    
    # Assign event based on task and trigger
    if task == 'Rest_GoNoGo':
        # Define Rest events if any, but according to user, rest is just a period
        # If no specific rest events, skip
        continue
    
    elif task == 'GoNoGo':
        # Triggers 1-4
        if trigger in [1, 2, 3, 4]:
            event_name = f'GoNoGo_Trigger_{trigger}'
            event_id = trigger  # 1-4 as per mapping
            task_specific_events.append([sample, 0, event_id])
    
    elif task == 'LandoitC':
        # Landoit-C triggers as per user's list
        # Memory Task Conditions: 1-50
        if trigger in [1,2,3,4,6,7,8,9,10,11,13,14,15,16,17,18,21,22,23,24,25,26,27,28,30,31,32,34,37,38,39,40,41,44,45,46,47,48,50]:
            # Use trigger as is for Memory Task Conditions
            event_id = trigger  # 1-50
            task_specific_events.append([sample, 0, event_id])
        elif trigger in [51, 52, 53, 54, 64, 128]:
            event_id = trigger
            task_specific_events.append([sample, 0, event_id])
        else:
            # Undefined trigger for Landoit-C
            logging.warning(f"Undefined Landoit-C trigger {trigger} at sample {sample}. Skipping.")
            continue
    
    elif task == 'MentalImagery':
        # Triggers 1-4
        if trigger in [1, 2, 3, 4]:
            event_name = f'MentalImagery_Trigger_{trigger}'
            event_id = trigger  # 1-4 as per mapping
            task_specific_events.append([sample, 0, event_id])

# Convert to numpy array
task_specific_events = np.array(task_specific_events)
logging.info(f"Number of task-specific events: {len(task_specific_events)}")

# ----------------------------
# Define Event ID Dictionaries
# ----------------------------

# Define separate event_id dictionaries for each task
EVENT_ID_GoNoGo = {
    'GoOnset': 1,
    'NoGoOnset': 2,
    'CorrectAnswer': 3,
    'WrongAnswer': 4
}

EVENT_ID_LandoitC = {
    **{f'MemoryCondition_{i}': i for i in [1,2,3,4,6,7,8,9,10,11,13,14,15,16,17,18,21,22,23,24,25,26,27,28,30,31,32,34,37,38,39,40,41,44,45,46,47,48,50]},  # 1-50
    'ResponseCorrect': 51,
    'ResponseIncorrect': 52,
    'NoResponse': 54,
    'AudioDistraction': 64,
    'PainDistraction': 128
}

EVENT_ID_MentalImagery = {
    'RightFistImagery': 1,
    'LeftFistImagery': 2,
    'RightFistReal': 3,
    'LeftFistReal': 4
}

# ----------------------------
# 9. Create Epochs for Each Dataset
# ----------------------------

# Set a common epoch time window, e.g., -0.2s to 0.5s around the event
tmin = -0.2
tmax = 0.8
# Create epochs for Go/No-Go Task
epochs_gonogo = mne.Epochs(raw, events=task_specific_events, event_id=EVENT_ID_GoNoGo,
                           tmin=tmin, tmax=tmax, preload=True, baseline=(None, 0),
                           reject=dict(eeg=150e-6))  # Adjust rejection thresholds as needed
logging.info(f"Number of Go/No-Go epochs: {len(epochs_gonogo)}")

# Create epochs for Landoit-C Task
# Separate Memory Task Conditions and Other Events
memory_conditions = {f'MemoryCondition_{i}': i for i in [1,2,3,4,6,7,8,9,10,11,13,14,15,16,17,18,21,22,23,24,25,26,27,28,30,31,32,34,37,38,39,40,41,44,45,46,47,48,50]}
landoitc_other_events = {k: v for k, v in EVENT_ID_LandoitC.items() if not k.startswith('MemoryCondition')}

# Epochs for Memory Task Conditions
epochs_landoitc_memory = mne.Epochs(raw, events=task_specific_events, event_id=memory_conditions,
                                   tmin=tmin, tmax=tmax, preload=True, baseline=(None, 0),
                                   reject=dict(eeg=150e-6))
logging.info(f"Number of Landoit-C Memory Task epochs: {len(epochs_landoitc_memory)}")

# Epochs for Other Landoit-C Events
epochs_landoitc_other = mne.Epochs(raw, events=task_specific_events, event_id=landoitc_other_events,
                                   tmin=tmin, tmax=tmax, preload=True, baseline=(None, 0),
                                   reject=dict(eeg=150e-6))
logging.info(f"Number of Landoit-C Other epochs: {len(epochs_landoitc_other)}")

# Create epochs for Mental Imagery Task
epochs_mentalimagery = mne.Epochs(raw, events=task_specific_events, event_id=EVENT_ID_MentalImagery,
                                  tmin=tmin, tmax=tmax, preload=True, baseline=(None, 0),
                                  reject=dict(eeg=150e-6))
logging.info(f"Number of Mental Imagery epochs: {len(epochs_mentalimagery)}")
